In [ ]:

from flask import Flask, render_template, redirect, url_for, session, flash, request, jsonify, make_response
from flask_sqlalchemy import SQLAlchemy
from flask_wtf import FlaskForm
from wtforms import StringField, PasswordField, SubmitField, BooleanField, TextAreaField
from wtforms.validators import InputRequired, Email, Length, ValidationError
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user
from flask_bcrypt import Bcrypt
import pandas as pd
import sqlite3
import jwt
import datetime
from functools import wraps

app = Flask(__name__)

db = SQLAlchemy(app)
bcrypt = Bcrypt(app)
app.config['SQLALCHEMY_DATABASE_URI'] = app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///userdata.db'
app.config['SECRET_KEY'] = 'thisisasecretkey'

class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(50))
    password = db.Column(db.String(80))
    admin = db.Column(db.Boolean)


class Liveshow(db.Model):
    showid = db.Column(db.Integer, primary_key=True)
    style = db.Column(db.Text)
    title = db.Column(db.Text)
    link = db.Column(db.Text)
    src = db.Column(db.Text)
    artist = db.Column(db.Text)
    time = db.Column(db.Text)
    price= db.Column(db.Text)
    address = db.Column(db.Text)
    
def token_required(f):
    @wraps(f)
    def decorated(*args, **kwargs):
        token = None

        if 'x-access-token' in request.headers:
            token = request.headers['x-access-token']

        if not token:
            return jsonify({'message' : 'Token is missing!'}), 401

        try: 
            data = jwt.decode(token, app.config['SECRET_KEY'],algorithms='HS256')
            current_user = User.query.filter_by(id=data['id']).first()
        except:
            return jsonify({'message' : 'Token is invalid!'}), 401

        return f(current_user, *args, **kwargs)

    return decorated

@app.route('/register', methods=['POST'])
def register():
    data = request.get_json()
    hashed_password = bcrypt.generate_password_hash(data['password'])
    new_user = User(name=data['name'], password=hashed_password, admin=False)
    db.session.add(new_user)
    db.session.commit()
    return jsonify({'message' : 'Registration completed!'})


@app.route('/login')
def login():
    auth = request.authorization

    if not auth or not auth.username or not auth.password:
        return make_response('Could not verify', 401, {'WWW-Authenticate' : 'Basic realm="Login required!"'})

    user = User.query.filter_by(name=auth.username).first()

    if not user:
        return make_response('Could not verify', 401, {'WWW-Authenticate' : 'Basic realm="Login required!"'})

    if bcrypt.check_password_hash(user.password, auth.password):
        token = jwt.encode({'id' : user.id, 'exp' : datetime.datetime.utcnow() + datetime.timedelta(minutes=30)}, app.config['SECRET_KEY'])

        return jsonify({'token' : token})

    return make_response('Could not verify', 401, {'WWW-Authenticate' : 'Basic realm="Login required!"'})
    return render_template('login.html', title="Login", form=form)

@app.route('/user', methods=['GET'])
@token_required
def get_all_users(current_user):

    if not current_user.admin:
        return jsonify({'message' : 'Cannot perform that function!'})

    users = User.query.all()

    output = []

    for user in users:
        user_data = {}
        user_data['name'] = user.name
        user_data['password'] = str(user.password)
        user_data['admin'] = user.admin
        output.append(user_data)

    return jsonify({'users' : output})

@app.route('/timetable', methods=['GET'])
@token_required
def get_all_shows(current_user):
    
    liveshows = Liveshow.query.all()

    output = []

    for liveshow in liveshows:
        liveshow_data = {}
        liveshow_data['showid'] = liveshow.showid
        liveshow_data['style'] = liveshow.style
        liveshow_data['title'] = liveshow.title
        liveshow_data['link'] = liveshow.link
        liveshow_data['src'] = liveshow.src
        liveshow_data['artist'] = liveshow.artist
        liveshow_data['time'] = liveshow.time
        liveshow_data['price'] = liveshow.price
        liveshow_data['address'] = liveshow.address
        
        output.append(liveshow_data)

    return jsonify({'liveshows' : output})

@app.route('/liveshow', methods=['POST'])
@token_required
def create_show(current_user):
    data = request.get_json()

    new_show = Liveshow(style=data['style'],title=data['title'], link=data['link'],src=data['src'],artist=data['artist'],time=data['time'],price=data['price'],address=data['address'],)
    db.session.add(new_show)
    db.session.commit()

    return jsonify({'message' : "new liveshow created!"})

@app.route('/liveshow/<showid>', methods=['DELETE'])
@token_required
def delete_show(current_user, showid):
    if not current_user.admin:
        return jsonify({'message' : 'Cannot perform that function!'})

    liveshow = Liveshow.query.filter_by(showid=showid).first()

    if not liveshow:
        return jsonify({'message' : 'No liveshow found!'})

    db.session.delete(liveshow)
    db.session.commit()

    return jsonify({'message' : 'liveshow item deleted!'})


@app.errorhandler(404)
def page_not_found(e):
    return render_template('404.html'), 404


if __name__ == '__main__':
    app.run('localhost', 9079)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


/Users/guitianshu/opt/anaconda3/lib/python3.8/site-packages/flask_sqlalchemy/__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(
 * Running on http://localhost:9079/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Jan/2022 17:31:30] "DELETE /liveshow/5 HTTP/1.1" 200 -
